In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score, precision_score, balanced_accuracy_score
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [ ]:
import wandb
!wandb login 5a28b268df4c40793395a35db3935741e6f49711

In [2]:
device

device(type='cuda', index=0)

In [3]:
# transforms_train = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=1),
#     transforms.Resize((350, 180)),
#     transforms.Grayscale(num_output_channels=1),
#     transforms.ToTensor()
# ])
# transforms_test = transforms.Compose([
#     transforms.Resize((350, 180)),
#     transforms.Grayscale(num_output_channels=1),
#     transforms.ToTensor()
# ])
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
    transforms.Resize((170, 180)),
    # transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])
transforms_test = transforms.Compose([
    transforms.Resize((170, 180)),
    # transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [4]:
# train_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/train/"
# test_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/test/"
# val_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/val/"
#
# train_classa_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/train/absent/"
# train_classb_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/train/present/"
# # test_classa_dir = "/home/katko/cropped_grayscale_mmodes_dataset/test/absent/"
# test_classa_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/test/absent/"
# test_classb_dir = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/test/present/"

In [5]:
train_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/test_new_minus_pleura_dataset/train_/"
test_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/test_new_minus_pleura_dataset/test_/"
val_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/test_new_minus_pleura_dataset/val_/"

# train_classa_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/train/absent/"
# train_classb_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/train/present/"
# # test_classa_dir = "/home/katko/cropped_grayscale_mmodes_dataset/test/absent/"
# test_classa_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/test/absent/"
# test_classb_dir_new = "/home/katko/Documents/Katka/skola/bc/cvat_usg_lung/train_test_val_new_dataset_cropped/test/present/"

In [6]:
train_dataset = datasets.ImageFolder(train_dir_new, transforms_train)
test_dataset = datasets.ImageFolder(test_dir_new, transforms_test)
val_dataset = datasets.ImageFolder(val_dir_new, transforms_test)

In [7]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=4)

In [8]:
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

print(train_dataset[0][0][0,:,:].shape)
print(train_dataset[0][0][0,:,:])
print(train_dataset[0][1])

3276
479
940
torch.Size([170, 180])
tensor([[0.1804, 0.1804, 0.1804,  ..., 0.1725, 0.1765, 0.1765],
        [0.1255, 0.1255, 0.1255,  ..., 0.1373, 0.1373, 0.1373],
        [0.2235, 0.2235, 0.2118,  ..., 0.1765, 0.1804, 0.1804],
        ...,
        [0.1333, 0.1333, 0.1216,  ..., 0.1882, 0.1804, 0.1804],
        [0.2431, 0.2431, 0.2431,  ..., 0.1961, 0.1882, 0.1882],
        [0.2431, 0.2431, 0.2431,  ..., 0.1569, 0.1569, 0.1569]])
0


In [ ]:
for i in range(len(test_dataloader.dataset)):
    print(test_dataloader.dataset[i][1])

In [ ]:
val_dataloader.dataset[40][0].shape

In [ ]:
plt.imshow(train_dataloader.dataset[340][0][0,:,:], cmap='gray')

In [ ]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
print('Val dataset size:', len(val_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams.update({'font.size': 20})
def imshow(input, title):
    # torch.Tensor => numpy
    print(input.shape)
    input = input.numpy().transpose((1, 2, 0))
    print(input.shape)
    # input = np.clip(input, 0, 1)
    # print(input.shape)
    plt.imshow(input)
    plt.title(title)
    plt.show()
# load a batch of train image
iterator = iter(train_dataloader)
# visualize a batch of train image
inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs[:8])
imshow(out, title=[class_names[x] for x in classes[:8]])

In [ ]:
class GrayscaleResnetTest(nn.Module):
    def __init__(self, in_channels=1, out_channels=2, pretrained=True, dropout_prob=0.8):
        super(GrayscaleResnetTest, self).__init__()
        self.model = torchvision.models.resnet18(pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # self.model.fc = nn.Sequential(
        #     nn.Dropout(p=dropout_prob),
        #     nn.Linear(self.model.fc.in_features, out_channels, bias=True)
        # )

    def forward(self, x):
        return self.model(x)

In [ ]:
resnet = GrayscaleResnetTest()
input = torch.randn((16,1,350,180))
output = resnet(input)
print(output.shape)

In [ ]:
model = models.resnet18(pretrained=True)
print(model)
# torch.cuda.empty_cache()
# num_features = model.fc.in_features     #extract fc layers features
# model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()  #(set loss function)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        return x

class GrayscaleResnetTest_v2(nn.Module):
    def __init__(self):
        super(GrayscaleResnetTest_v2, self).__init__()
        self.model = ResNet(BasicBlock, [2, 2, 2])

    def forward(self, x):
        x = self.model(x)
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.size(0), -1)
        return x

In [ ]:
model = GrayscaleResnetTest()
# model = GrayscaleResnetTest_v2()
# print(model)
torch.cuda.empty_cache()

model = model.to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-08)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
print(list(model.children()))

In [ ]:
wandb.init(project="classification-lungsliding-pa-newdataset")

num_epochs = 16
start_time = time.time()
for epoch in range(num_epochs):
    print("Epoch {} running".format(epoch))
    """ Training Phase """
    model.train()
    running_loss = 0.   #(set loss 0)
    running_corrects = 0
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        # print("INPUTS SHAPE ->" + str(inputs.size()))
        # conv = torch.nn.Conv2d(3, 1, 1)
        # inputs = conv(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)
        torch.autograd.set_detect_anomaly(False)

        # print("##### LABELS #####")
        # print(labels)

        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        # print("OUT SHAPE ->" + str(outputs.shape))
        _, preds = torch.max(outputs, 1)

        # print("##### PREDS #####")
        # print(preds)

        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)
    # wandb.log({"Train loss": loss, "Train acc":epoch_acc})
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))

    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in val_dataloader:
            # conv = torch.nn.Conv2d(3, 1, 1)
            # print(conv(inputs).size())
            # print(inputs.size(dim=1))
            # inputs = conv(inputs)
            # print(inputs.shape)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset)

        wandb.log({"Test loss": loss, "Test acc":epoch_acc})

        print('[Test #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

In [ ]:
model = models.resnet18(pretrained=True)
# model = GrayscaleResnetTest()
def append_dropout(model, rate=0.8):
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            append_dropout(module)
        if isinstance(module, nn.ReLU):
            new = nn.Sequential(module, nn.Dropout2d(p=rate, inplace=False))
            setattr(model, name, new)

append_dropout(model)

In [ ]:
print(model)

In [ ]:
##########################################################################################################################################
##################################################### TEST 2 #############################################################################
##########################################################################################################################################
# model = GrayscaleResnetTest()
# model = GrayscaleResnetTest_v2()
# wandb.watch(model)
# print(model)
torch.cuda.empty_cache()

model = model.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
# optimizer = optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, eps=1e-08)
optimizer = optim.Adam(model.parameters(), lr=lr)
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

num_epochs = 60
start_time = time.time()

run = wandb.init(project="classificator-videosplit_v1",
           config={
                   "start_time": start_time,
                   "layers":3,
                   "optimizer": optimizer,
                   "learning_rate": lr,
                   "loss":criterion,
                   "num_epochs":num_epochs
           } )
for epoch in range(num_epochs):
    print("Epoch {} running".format(epoch))

    """ Training Phase """
    model.train()
    running_loss = 0.
    running_corrects = 0

    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)

    """ Validation Phase """
    model.eval()
    val_running_loss = 0.
    val_running_corrects = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            val_running_loss += criterion(outputs, labels).item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = val_running_corrects / len(val_dataset)

    # Calculate the balanced accuracy, F1 score, and precision
    bacc = balanced_accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    wandb.log({"Training Loss": epoch_loss, "Training Accuracy":epoch_acc, "Validation Loss":val_epoch_loss, "Validation Accuracy":val_epoch_acc, "Balanced Accuracy":bacc, "F1 Score":f1, "Precision":precision})
    print('Epoch: {} | Training Loss: {:.4f} | Training Accuracy: {:.4f} | Validation Loss: {:.4f} | Validation Accuracy: {:.4f} | Balanced Accuracy: {:.4f} | F1 Score: {:.4f} | Precision: {:.4f}'.format(epoch+1, epoch_loss, epoch_acc, val_epoch_loss, val_epoch_acc, bacc, f1, precision))

In [ ]:
save = 'finalasimodel.pth'
torch.save(model.state_dict(), save)

In [ ]:
 # """ Testing Phase """
model.load_state_dict(torch.load('model_adam001_60.pth'))
print(save)
model.eval()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        print(labels)
        print(preds)
        print("_______________________________________________________________")
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset)
    print('Test Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time()- start_time))

In [ ]:
import sklearn.metrics as metrics

# """ Testing Phase """
model.load_state_dict(torch.load(save))
# print(save)
model.eval()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    all_labels = []
    all_preds = []
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset)
    epoch_f1 = metrics.f1_score(all_labels, all_preds, average='macro')
    epoch_balanced_acc = metrics.balanced_accuracy_score(all_labels, all_preds)
    epoch_precision = metrics.precision_score(all_labels, all_preds, average='macro')
    print('Test Loss: {:.4f} | Test Acc: {:.4f} | F1: {:.4f} | Balanced Acc: {:.4f} | Precision: {:.4f}'.format(epoch_loss, epoch_acc, epoch_f1, epoch_balanced_acc, epoch_precision))


In [ ]:
inputs, labels = test_dataloader.dataset[24]

inputs = inputs.to(device)
labels = torch.tensor([labels]).to(device)
inputs = inputs.unsqueeze(0)
inputs.requires_grad = True
print("##### LABELS #####")
print(labels)
# forward inputs and get output
outputs = model(inputs)
print("OUT SHAPE ->" + str(outputs.shape))
_, preds = torch.max(outputs, 1)
print("##### PREDS #####")
print(preds)
loss = criterion(outputs, labels)
# get loss value and update the network weights
loss.backward()
plt.imshow(inputs.grad[0].cpu().detach().permute(1,2,0))

In [ ]:
plt.imshow((inputs.grad[0].cpu().detach().permute(1,2,0) - inputs.grad[0].cpu().detach().permute(1,2,0).min()) / (inputs.grad[0].cpu().detach().permute(1,2,0) - inputs.grad[0].cpu().detach().permute(1,2,0).min()) .max())

In [ ]:
plt.imshow(inputs.grad[0].cpu().detach().permute(1,2,0).abs() /  (inputs.grad[0].cpu().detach().permute(1,2,0).abs().max()))

In [ ]:
plt.imshow(inputs.grad[0].cpu().detach().permute(1,2,0).abs() /  (inputs.grad[0].cpu().detach().permute(1,2,0).abs().max()))

In [ ]:
##Testing
model.eval()
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for i, (inputs, labels) in enumerate(val_dataloader):
        print(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        print(preds)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        if i == 0:
            print('======>RESULTS<======')
            images = torchvision.utils.make_grid(inputs[:4])
            imshow(images.cpu(), title=[class_names[x] for x in labels[:4]])
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset)
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.
          format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

In [ ]:
class GrayscaleResnet(nn.Module):
    def __init__(self, in_channels=1,out_channels=2,pretrained=False):
        super(GrayscaleResnet, self).__init__()
        self.model = torchvision.models.resnet18(pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(in_channels,64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features,out_channels)
        print(in_channels)
        print(out_channels)
        print(pretrained)
        print(self.model.fc)
    def forward(self, x):
        return self.model(x)

In [ ]:
model = GrayscaleResnet()
wandb.watch(model, log_freq=10) #load resnet18 model
print(model)
torch.cuda.empty_cache()
model = model.to(device)
criterion = nn.CrossEntropyLoss()  #(set loss function)
# optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-08)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# num_epochs = 100
# start_time = time.time() #(for showing time)
# for epoch in range(num_epochs):
#     print("Epoch {} running".format(epoch))
#     """ Training Phase """
#     model.train()
#     running_loss = 0.   #(set loss 0)
#     running_corrects = 0
#     # load a batch data of images
#     for i, (inputs, labels) in enumerate(train_dataloader):
#         # print(inputs[0][0,:,:])
#         # print(inputs[0][0])
#         # inputs = inputs[0][0,:,:]
#         # print("INPUTS SHAPE ->" + str(inputs.size()))
#         # inputs.size(dim=1) = 1
#         conv = torch.nn.Conv2d(3, 1, 1)
#         # print(conv(inputs).size())
#         # print(inputs.size(dim=1))
#         inputs = conv(inputs)
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         # print(labels)
#         torch.autograd.set_detect_anomaly(False)
#
#
#         print("##### LABELS #####")
#         print(labels)
#         # forward inputs and get output
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print("OUT SHAPE ->" + str(outputs.shape))
#         _, preds = torch.max(outputs, 1)
#         print("##### PREDS #####")
#         print(preds)
#         loss = criterion(outputs, labels)
#         # get loss value and update the network weights
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)
#
#     epoch_loss = running_loss / len(train_dataset)
#     epoch_acc = running_corrects / len(train_dataset)
#     # train_losses.append(np.array(epoch_acc))
#     print('[Train #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
#
#     """ Testing Phase """
#     model.eval()
#     with torch.no_grad():
#         running_loss = 0.
#         running_corrects = 0
#         for inputs, labels in test_dataloader:
#             conv = torch.nn.Conv2d(3, 1, 1)
#             # print(conv(inputs).size())
#             # print(inputs.size(dim=1))
#             inputs = conv(inputs)
#             inputs = inputs.to(device)
#             labels = labels.to(device)
#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)
#             running_loss += loss.item() * inputs.size(0)
#             running_corrects += torch.sum(preds == labels.data)
#         epoch_loss = running_loss / len(test_dataset)
#         epoch_acc = running_corrects / len(test_dataset)
#         # val_losses.append(np.array(epoch_acc))
#
#         print('[Test #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

In [ ]:
# early stop

In [ ]:
# num_epochs = 30   #(set no of epochs)
# start_time = time.time() #(for showing time)
# for epoch in range(num_epochs): #(loop for every epoch)
#     print("Epoch {} running".format(epoch)) #(printing message)
#     """ Training Phase """
#     model.train()    #(training model)
#     running_loss = 0.   #(set loss 0)
#     running_corrects = 0
#     # load a batch data of images
#     for i, (inputs, labels) in enumerate(train_dataloader):
#         # print("INPUTS SHAPE ->" + str(inputs.shape))
#         inputs.requires_grad=True
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         # print("##### LABELS #####")
#         # print(labels)
#         # forward inputs and get output
#         # optimizer.zero_grad()
#         outputs = model(inputs)
#         # print("OUT SHAPE ->" + str(outputs.shape))
#         _, preds = torch.max(outputs, 1)
#         # print("##### PREDS #####")
#         # print(preds)
#         loss = criterion(outputs, labels)
#         # get loss value and update the network weights
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)
#         print(inputs.grad.shape)
#         break
    # epoch_loss = running_loss / len(train_dataset)
    # epoch_acc = running_corrects / len(train_dataset)
    # print('[Train #{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))

    # """ Testing Phase """
    # model.eval()
    # with torch.no_grad():
    #     running_loss = 0.
    #     running_corrects = 0
    #     for inputs, labels in test_dataloader:
    #         inputs = inputs.to(device)
    #         labels = labels.to(device)
    #         outputs = model(inputs)
    #         _, preds = torch.max(outputs, 1)
    #         loss = criterion(outputs, labels)
    #         running_loss += loss.item() * inputs.size(0)
    #         running_corrects += torch.sum(preds == labels.data)
    #     epoch_loss = running_loss / len(test_dataset)
    #     epoch_acc = running_corrects / len(test_dataset) * 100.
    #     print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))